In [19]:
import os

In [1]:
class Hooklog:
    path = None
    par = None
    
    digitname = None
    length = None
    li = None
    
    def __init__(self, path, par):
        self.path = path
        self.par = par
        self.li = list()
        self.length = 0
        
        self.__parseDigitname()
        self.__parseHooklog()
        
    def isExist(self):
        if os.path.isfile(self.path):
            return True
        else:
            return False
    
    def __len__(self):
        return self.length

    def __iter__(self):
        return iter(self.li)
    
    def __getitem__(self, key):
        return self.li[key]
    
    def __str__(self):
        return "class Hooklog, %s, par = %d, len = %d, digit name = %s" % (self.path, self.par, 
                                                                           self.length, self.digitname)
    
    def __parseDigitname(self):
        if self.path != "":
            self.digitname = self.path.split('/')[-1][:6] + '-' + self.path.split('_')[-1].split('.')[0]
    
    # private functions
    def __parseHooklog(self):
        if self.isExist():
            handle = open(self.path, 'rb')
            
            while 1:
                (tick, api) = self.__getNextPair(handle)
                if tick == 0:
                    break # end of file or fail to read
                self.li.append((tick, api))
                self.length += 1               #------why directly -> self.length = len(self.li)
            if handle: handle.close()
            
            self.li.sort(key = lambda tup: tup[0]) # sort by tick
        else:
            pass
    
    def __getNextPair(self, handle):
        tick = 0
        api = ''
        
        while 1:
            line = handle.readline()
            if not line: break # end of file
            if line[0] == '#': # start a new call
                tick = line[1:].strip()
                api = handle.readline().strip()
                
                # hack
                if api.startswith('=Reg'): api = api[1:]
                
                if self.par == True:
                    api = self._getParValue(api, handle)
                break # get one call
            else:
                continue
        
        return (tick, api)
    
    # protected functions
    # MIKE: 20160307, basic par. I move it out for better creating new class FeatureHooklog. 
    #                            It has to be protected!
    def _getParValue(self, api, handle):
        if api.startswith('Reg') or api == 'LoadLibrary':
            api += handle.readline().strip()
            return api
    
    # public functions
    def getWinSet(self, win):
        li_set = set()
        for i in range(self.length - win + 1): #----for i in range(9)
            key = ''
            for w in range(win):               #----for w in range(3)
                if (self.li[i+w][1] != None):
                    key += self.li[i+w][1]
                else:
                    key += "None"
#                 key += self.li[i+w][1] # api   #----if none -> type error
            li_set.add(key)
        return li_set

In [2]:
# h1= Hooklog("data/hooklogs/0e153c188c76206a672014b7_992.trace.hooklog", True)
# h1= Hooklog("../data/hooklogs/02c27a2e4c215d22fabdfc7b46fd6f20_3132.trace.hooklog", True)
# print "len: ",h1.__len__()
# print "list: ",list(h1.__iter__())
# print "item: ",h1.__getitem__(3)
# print "Win: ",h1.getWinSet(1)

In [22]:
class FeatureHooklog(Hooklog):
    
    def __init__(self, path):
        Hooklog.__init__(self, path, 1)

    def __str__(self):
        return "class FeatureHooklog, %s, par = %d, len = %d, digit name = %s" % (self.path, self.par, self.length, self.digitname)
    
    # protected functions
    # MIKE: 20160307, in FeatureHooklog, new api is (api,firstpar,return)
    def _getParValue(self, api, handle):                    #------handle?? input??
        if api.startswith('Reg'):
            v = handle.readline().strip().lower().split('=')[-1]
            return (api, self.__keyTrans(v), self.__getReturn(handle))
        elif api == 'LoadLibrary':
            return (api, self.__libTrans(v, "SYS"), r)
        elif api.endswith('File'):
            v = handle.readline().strip().lower().split('=')[-1]
            return (api, self.__libTrans(v, "LOC"), self.__getReturn(handle))
        elif api == 'OpenProcess':
            r = self.__getReturn(handle)
            v = handle.readline().strip().lower().split('=')[-1] if r == 1 else ""
            return (api, v, r)
        elif api == 'ExitProcess':
            return (api, "", 1)
        else: # MIKE: process-related
            v = handle.readline().strip().lower().split('=')[-1]
            return (api, self.__libTrans(v, "LOC"), self.__getReturn(handle))
    
    # private functions
    def __libTrans(self, value, loc_file):
        DIR = LIB = EXT = "NON"
        
        if value == "":
            return "NON\\NON\\NON"
        
        tokens = value.split('\\')
        
        # DIR
        if value[0:2] == '\\\\':
            DIR = "SMB"
        elif len(tokens) == 1:
            DIR = loc_file # 1) for file, it is "LOC"; for library, it is "SYS".
        elif len(tokens) == 2 and value[1] == ':':
            DIR = "ROT"
        else:
            for (d,s) in dir_list:
                if d in value:
                    DIR = s
                    break

        # EXT
        t = tokens[-1].split('.')
        ext = t[-1]
        if ext == "" or len(t) == 1: # extension is . or no extension
            EXT = "NON"
        else:
            EXT = ext
            
        # LIB
        LIB = tokens[-1].split('.')[0]
        
        return DIR+"\\"+LIB+"."+EXT
    
    
    def __keyTrans(self, value):
        HK = KEY = "NON"
        if value == "": return "NON@NON"
        
        tokens = value.split('\\')
        
        # HK
        if tokens[0] in hkey_dict:
            HK = hkey_dict[tokens[0]]
        else:
            HK = "SUB"
            
        # KEY
        subkey = value[value.find('\\'):]
        KEY = self.__removeId(subkey)
        
        return HK+"@"+KEY
    
    def __getReturn(self, handle):
        while 1:
            r = handle.readline()
            if r[0:6] =='Return' or r[0:6] =='result':
                return 1 if r[7] == 'S' else -1
            
    def __removeId(self, key):
        rvalue = ""
        
        for t in key.split('\\'):
            if t.count('-') > 3:
                rvalue += "\\CID" # CLSID
            elif t.isdigit():
                rvalue += "\\DIG"
            else:
                rvalue += ('\\' + t)
        return rvalue

    # public functions
    def getWinSet(self, win):
        li_set = set()
        for i in range(self.length - win + 1):
            key = list()
            for w in range(win):
                key.append(self.li[i+w][1]) # append (api, v, r)
            li_set.add(tuple(key))
        return li_set
    
# Ordered!!
dir_list = [
    ("\\windows\\temp\\", "TMP"),
    (":\\temp\\", "TMP"),
    ("\\windows\\system32\\", "SYS"),
    ("\\windows\\system\\", "SYS"),
    ("\\windows\\", "WIN"),
    ("\\program files\\", "PRO"),
    ("\\documents and settings\\", "USR"),
    ("\\docume~1\\", "USR")
]

hkey_dict = {
    "hkey_classes_root": "HKCR",
    "hkey_current_user": "HKCU",
    "hkey_local_machine": "HKLM",
    "hkey_users": "HKUS",
    "hkey_current_config": "HKCC"
}

In [23]:
# fhl = FeatureHooklog("data/hooklog/0e153c188c76206a672014b7_992.trace.hooklog")
# print fh1._getParValue('LoadLibrary',)


In [24]:
class FindHooklog:
    data_path = None
    proc_list = None
    ivl = None
    leaves = None
    
    def __init__(self, data_path, proc_list, ivl, leaves):
        self.data_path = data_path
        self.proc_list = proc_list
        self.ivl = ivl
        self.leaves = leaves
    
    def digit2hooklog(self, digit):
        idx = self.ivl.index(digit)
        idx = self.leaves[idx]
        return self.proc_list[idx]

    def hooklog2digit(self, hooklog):
        idx = self.proc_list.index(hooklog)
        idx = self.leaves.index(idx)
        return self.ivl[idx]

In [25]:
fl=("data/hooklog/0e153c188c76206a672014b7_992.trace.hooklog",)

In [26]:
class HooklogXml:
    path = None    
    length = None
    li = None
    
    def __init__(self, path):
        self.path = path
        self.li = list()
        self.length = 0
        
        self.__parseHooklogXml()

    def isExist(self):
        if os.path.isfile(self.path): return True
        else: return False
    
    def __len__(self):
        return self.length
    
    # private functions
    def __parseHooklogXml(self):
        if self.isExist():
            handle = open(self.path, 'rb')
            
            while 1:
                (tick, api, key_val_list) = self.__getNextCallXml(handle)
                if tick == 0:
                    break # end of file or fail to read
                self.li.append((tick, api, key_val_list))
                self.length += 1
            if handle: handle.close()
            
            self.li.sort(key = lambda tup: tup[0]) # sort by tick
        else:
            pass
    
    def __getNextCallXml(self, handle):
        tick = 0
        api = ''
        key_val_list = list()
        
        while 1:
            line = handle.readline()
            if not line: break # end of file
            if line[0] == '#': # start a new call
                tick = line[1:].strip()
                api = handle.readline().strip()
                
                # hack
                if api.startswith('=Reg'): api = api[1:]
                
                # rest of parameters
                while 1:
                    pos = handle.tell()
                    key_value = handle.readline().strip()
                    if not key_value: break
                    if key_value[0] == '#': # ouch
                        handle.seek(pos)
                        break
                    else:
                        key_val_list.append(key_value)
                break # get one call
            else:
                continue
        
        return (tick, api, key_val_list)
    
    # public functions
    def printXml(self):
        print "<?xml version=\"1.0\"?>"
        print "<Profile>"
        print "<Meta>"
        print "<Hash>%s</Hash>" % (self.path.split('_')[0].split('/')[-1])
        print "<Process_Id>%s</Process_Id>" % (self.path.split('_')[1].split('.')[0])
        print "</Meta>"
        
        print "<Execution>"
        for i in range(self.length):
            (tick, api, key_val_list) = self.li[i]
            o = "<" + api
            for k_v in key_val_list:
                tokens = k_v.split('=')
                if len(tokens) > 2: print "ERROR!!! Has a = in value."
                k = tokens[0]
                v = tokens[1]
                o += (" "+k+"=\""+v+"\"")
            o += " Time=\""+ tick +"\" />"
            print o
        print "</Execution>"
        print "</Profile>"

In [27]:
# hlx = HooklogXml("data/hooklog/0e153c188c76206a672014b7_992.trace.hooklog")
# print "printXml ", hlx.printXml() 

In [28]:
# hl = Hooklog("data/hooklog/0e153c188c76206a672014b7_992.trace.hooklog", True)
# print "hT: ", hl

# h2 = Hooklog("data/hooklog/0e153c188c76206a672014b7_992.trace.hooklog", False)
# print "hF: ", h2

# fhl = FeatureHooklog("data/hooklog/0e153c188c76206a672014b7_992.trace.hooklog")
# print "fhl: ", fhl

# hlx = HooklogXml("data/hooklog/0e153c188c76206a672014b7_992.trace.hooklog")
# print "hlx: ",hlx